In [1]:
# ====== BLOK 1 (VRAM-safe + limiter & schedule cepat) ======
import os, json, math, random, glob, time, warnings, csv, platform
from pathlib import Path
from typing import List, Tuple

import numpy as np
import pandas as pd
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

from tqdm import tqdm
warnings.filterwarnings("ignore")

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if DEVICE.type == "cuda":
    torch.backends.cudnn.benchmark = True
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    try: torch.set_float32_matmul_precision("high")
    except: pass

IS_WIN = (platform.system().lower() == "windows")
try:
    from IPython import get_ipython
    IN_NOTEBOOK = get_ipython() is not None
except Exception:
    IN_NOTEBOOK = False

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if DEVICE.type == "cuda": torch.cuda.manual_seed_all(SEED)

# ---- Paths (Windows) ----
BASE_WIN = r"C:\Python\Template\Data Science\test\HOLOGY_UB_2025"
TRAIN_IMG_DIR = os.path.join(BASE_WIN, "train", "images")
TRAIN_LBL_DIR = os.path.join(BASE_WIN, "train", "labels")
TEST_IMG_DIR  = os.path.join(BASE_WIN, "test",  "images")
SAMPLE_SUB    = os.path.join(BASE_WIN, "sample_submission.csv")
OUT_DIR       = os.path.join(BASE_WIN, "out_csrnet_efficientb0")
if not os.path.exists(TRAIN_IMG_DIR):
    TRAIN_IMG_DIR = "/workspace/train/images"
    TRAIN_LBL_DIR = "/workspace/train/labels"
    TEST_IMG_DIR  = "/workspace/test/images"
    SAMPLE_SUB    = "/workspace/sample_submission.csv"
    OUT_DIR       = "/workspace/out_csrnet_efficientb0"
os.makedirs(OUT_DIR, exist_ok=True)

# ================== KUNCI PERCEPATAN ==================
# 1) Batasi steps per epoch (kurangi 380 → misal 160)
MAX_STEPS_PER_EPOCH = 160   # ubah 120–200 sesuai target waktu

# 2) Progressive resizing (awal cepat @512, akhir polish @640)
LOWRES_EPOCHS       = 15
LOWRES_LONG_SIDE    = 512
HIGHRES_LONG_SIDE   = 640   # final resolution

# 3) Freeze backbone di awal (hemat backward compute)
FREEZE_FRONT_EPOCHS = 3
# ======================================================

# Preset aman RTX 3050 4GB
STRIDE         = 16
DIVISIBLE_BY   = 32
MAX_LONG_SIDE  = HIGHRES_LONG_SIDE  # akan ditimpa per-epoch (lihat Blok 9)
BATCH_SIZE     = 8
ACCUM_STEPS    = 2
MICRO_CHUNK    = 2

# Hyperparams
EPOCHS       = 20
LR           = 1e-4
WEIGHT_DECAY = 1e-4
CLIP_GRAD    = 1.0

# Loss weights
KNN_K        = 3
BETA         = 0.3
SIGMA_MINMAX = (1.0, 8.0)
OT_WEIGHT_WARM    = 1.2
OT_WEIGHT_BASE    = 1.0
COUNT_WEIGHT_WARM = 2e-2
COUNT_WEIGHT_BASE = 2e-2
LOCAL_WEIGHT      = 5e-3
DENSITY_ALPHA     = 0.05

# Augment (geom; photometric di Blok 4)
SCALE_JITTER = (0.85, 1.20)
COLOR_JITTER = (0.1, 0.1, 0.1)
HFLIP_P      = 0.5

AMP_ENABLED     = (DEVICE.type == "cuda")
AMP_DEVICE_TYPE = "cuda"
LOG_CSV = os.path.join(OUT_DIR, "train_log.csv")

USE_TORCH_COMPILE = False  # untuk start cepat

print(f"Device: {DEVICE} | STRIDE={STRIDE} | SIZE(init)={MAX_LONG_SIDE} | BATCH={BATCH_SIZE} | ACCUM={ACCUM_STEPS} | CHUNK={MICRO_CHUNK} | MAX_STEPS/ep={MAX_STEPS_PER_EPOCH}")
print("done")


Device: cuda | STRIDE=16 | SIZE(init)=640 | BATCH=8 | ACCUM=2 | CHUNK=2 | MAX_STEPS/ep=160
done


In [2]:
# ====== BLOK 2 ======
def resize_keep_ar_and_pad(img: Image.Image, target_long_side=MAX_LONG_SIDE, divisible_by=DIVISIBLE_BY):
    w, h = img.size
    scale = min(target_long_side / max(h, w), 1.0)
    new_w, new_h = int(round(w * scale)), int(round(h * scale))
    img_resized = img.resize((new_w, new_h), Image.BICUBIC)
    pad_h = (divisible_by - (new_h % divisible_by)) % divisible_by
    pad_w = (divisible_by - (new_w % divisible_by)) % divisible_by
    canvas = Image.new("RGB", (new_w + pad_w, new_h + pad_h), (0, 0, 0))
    canvas.paste(img_resized, (0, 0))
    return canvas, scale, (0, 0)

def scale_and_offset_points(points: np.ndarray, scale: float, offset_xy=(0, 0)):
    if points.size == 0: return points
    offx, offy = offset_xy
    pts = points.copy().astype(np.float32)
    pts[:, 0] = pts[:, 0] * scale + offx
    pts[:, 1] = pts[:, 1] * scale + offy
    return pts

def downscale_points_for_stride(points: np.ndarray, stride: int):
    if points.size == 0: return points
    pts = points.copy().astype(np.float32)
    pts /= stride
    return pts

def pairwise_dist(a: np.ndarray, b: np.ndarray):
    aa = np.sum(a*a, axis=1, keepdims=True)
    bb = np.sum(b*b, axis=1, keepdims=True).T
    ab = a @ b.T
    d2 = np.maximum(aa + bb - 2*ab, 0.0)
    return np.sqrt(d2 + 1e-8)

def geometry_adaptive_sigma(points_out: np.ndarray, k=KNN_K, beta=BETA, clamp=SIGMA_MINMAX):
    n = points_out.shape[0]
    if n == 0: return np.array([], dtype=np.float32)
    if n == 1: return np.array([(clamp[0] + clamp[1]) * 0.5], dtype=np.float32)
    D = pairwise_dist(points_out, points_out) + np.eye(n)*1e6
    knn = np.sort(D, axis=1)[:, :k]
    s = knn.mean(axis=1) * beta
    s = np.clip(s, clamp[0], clamp[1]).astype(np.float32)
    return s

def draw_density_gaussians(h_out: int, w_out: int, points_out: np.ndarray, sigmas: np.ndarray):
    den = np.zeros((h_out, w_out), dtype=np.float32)
    for (x, y), s in zip(points_out, sigmas):
        if not (0 <= x < w_out and 0 <= y < h_out): continue
        rad = int(max(1, math.ceil(3.0 * float(s))))
        x1, x2 = max(0, int(x) - rad), min(w_out - 1, int(x) + rad)
        y1, y2 = max(0, int(y) - rad), min(h_out - 1, int(y) + rad)
        if x2 < x1 or y2 < y1: continue
        xs = np.arange(x1, x2 + 1, dtype=np.float32)
        ys = np.arange(y1, y2 + 1, dtype=np.float32)
        X, Y = np.meshgrid(xs, ys)
        g = np.exp(-((X - x)**2 + (Y - y)**2) / (2.0 * (s**2) + 1e-6))
        g_sum = g.sum()
        if g_sum > 0:
            g /= g_sum
            den[y1:y2+1, x1:x2+1] += g.astype(np.float32)
    return den


In [3]:
# ====== BLOK 3 ======
def list_train_items(img_dir: str, lbl_dir: str) -> List[Tuple[str, str]]:
    imgs = sorted(glob.glob(os.path.join(img_dir, "*.jpg")))
    items = []
    for img_path in imgs:
        base = os.path.splitext(os.path.basename(img_path))[0]
        lbl_path = os.path.join(lbl_dir, f"{base}.json")
        if os.path.exists(lbl_path):
            items.append((img_path, lbl_path))
    return items

def read_points_from_json(json_path: str) -> np.ndarray:
    with open(json_path, "r") as f:
        obj = json.load(f)
    pts_raw = obj.get("points", [])
    out = []
    for p in pts_raw:
        if isinstance(p, dict):
            x = p.get("x", p.get("X", p.get("cx", p.get("u"))))
            y = p.get("y", p.get("Y", p.get("cy", p.get("v"))))
        elif isinstance(p, (list, tuple)) and len(p) >= 2:
            x, y = p[0], p[1]
        else:
            continue
        try:
            x = float(x); y = float(y)
            if np.isfinite(x) and np.isfinite(y):
                out.append([x, y])
        except (TypeError, ValueError):
            continue
    if len(out) == 0:
        return np.zeros((0, 2), dtype=np.float32)
    return np.asarray(out, dtype=np.float32)


In [4]:
# ====== BLOK 4 — Dataset (LABEL-LIGHT, tanpa density GT di Dataset) ======
from PIL import ImageOps, ImageEnhance

IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]
IMG_NORM = transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)

# Photometric (dipersempit agar ringan)
PHOTOMETRIC_ENABLE     = True
GAMMA_JITTER_RANGE     = (0.90, 1.15)
EXPOSURE_JITTER_RANGE  = (0.95, 1.08)
GRAYSCALE_P            = 0.08
COLOR_JITTER_SAFE      = COLOR_JITTER
AUTOCONTRAST_P         = 0.00  # dimatikan demi kecepatan

def resize_keep_ar(img: Image.Image, target_long_side: int):
    w, h = img.size
    scale = min(target_long_side / max(h, w), 1.0)
    new_w, new_h = int(round(w * scale)), int(round(h * scale))
    img_resized = img.resize((new_w, new_h), Image.BICUBIC)
    return img_resized, scale

def pad_to_divisible(img: Image.Image, divisible_by: int = DIVISIBLE_BY):
    w, h = img.size
    pad_h = (divisible_by - (h % divisible_by)) % divisible_by
    pad_w = (divisible_by - (w % divisible_by)) % divisible_by
    if pad_h == 0 and pad_w == 0:
        return img, (0, 0)
    canvas = Image.new("RGB", (w + pad_w, h + pad_h), (0, 0, 0))
    canvas.paste(img, (0, 0))
    return canvas, (0, 0)

def transform_points(points: np.ndarray, scale: float, off=(0, 0)):
    if points.size == 0: return points
    pts = points.astype(np.float32).copy()
    pts[:, 0] = pts[:, 0] * scale + off[0]
    pts[:, 1] = pts[:, 1] * scale + off[1]
    return pts

def downscale_points_for_stride(points: np.ndarray, stride: int):
    if points.size == 0: return points
    return (points.astype(np.float32) / float(stride))

def apply_gamma(img: Image.Image, gamma: float) -> Image.Image:
    if abs(gamma - 1.0) < 1e-3: return img
    lut = [min(255, max(0, int((i / 255.0) ** gamma * 255.0 + 0.5))) for i in range(256)]
    if img.mode != "RGB": img = img.convert("RGB")
    r, g, b = img.split()
    r = r.point(lut); g = g.point(lut); b = b.point(lut)
    return Image.merge("RGB", (r, g, b))

def apply_exposure(img: Image.Image, gain: float) -> Image.Image:
    if abs(gain - 1.0) < 1e-3: return img
    return ImageEnhance.Brightness(img).enhance(gain)

def maybe_grayscale(img: Image.Image, p: float) -> Image.Image:
    if random.random() < p: return ImageOps.grayscale(img).convert("RGB")
    return img

class CrowdTrainDataset(Dataset):
    def __init__(self, items: List[Tuple[str, str]], train: bool = True):
        self.items = items
        self.train = train
        self.to_tensor = transforms.ToTensor()
        self.normalize = IMG_NORM
        self.color_jitter = transforms.ColorJitter(
            brightness=COLOR_JITTER_SAFE[0],
            contrast=COLOR_JITTER_SAFE[1],
            saturation=COLOR_JITTER_SAFE[2]
        )

    def __len__(self): return len(self.items)

    def __getitem__(self, idx):
        img_path, lbl_path = self.items[idx]
        img = Image.open(img_path).convert("RGB")
        points = read_points_from_json(lbl_path)  # numpy (N,2) pada koordinat asli

        # scale jitter
        scale_long = int(round(MAX_LONG_SIDE * (random.uniform(*SCALE_JITTER) if self.train else 1.0)))
        img_r, s = resize_keep_ar(img, scale_long)
        points_r = transform_points(points, s, (0, 0))

        # flip + photometric (ringan)
        if self.train:
            if random.random() < HFLIP_P:
                img_r = img_r.transpose(Image.FLIP_LEFT_RIGHT)
                w = img_r.size[0]
                if points_r.size > 0:
                    pts = points_r.copy()
                    pts[:, 0] = (w - 1) - pts[:, 0]
                    points_r = pts
            if PHOTOMETRIC_ENABLE:
                img_r = apply_gamma(img_r, random.uniform(*GAMMA_JITTER_RANGE))
                img_r = apply_exposure(img_r, random.uniform(*EXPOSURE_JITTER_RANGE))
                img_r = maybe_grayscale(img_r, GRAYSCALE_P)
                img_r = self.color_jitter(img_r)

        # pad → kelipatan
        img_r, _ = pad_to_divisible(img_r, DIVISIBLE_BY)

        # points pada grid output
        points_out = downscale_points_for_stride(points_r, STRIDE)

        # tensor gambar
        img_t = self.normalize(self.to_tensor(img_r).float().contiguous())

        # LABEL-LIGHT: tidak membuat density di sini
        return {
            "image": img_t,
            "count": torch.tensor(float(points_out.shape[0]), dtype=torch.float32),
            "img_path": img_path,
            "points_out_np": points_out.astype(np.float32)
        }

class CrowdTestDataset(Dataset):
    def __init__(self, img_dir: str):
        self.imgs = sorted(glob.glob(os.path.join(img_dir, "*.jpg")))
        self.to_tensor = transforms.ToTensor()
        self.normalize = IMG_NORM

    def __len__(self): return len(self.imgs)

    def __getitem__(self, idx):
        img_path = self.imgs[idx]
        img = Image.open(img_path).convert("RGB")
        img_r, _ = resize_keep_ar(img, MAX_LONG_SIDE)
        img_r, _ = pad_to_divisible(img_r, DIVISIBLE_BY)
        w, h = img_r.size
        w_out, h_out = w // STRIDE, h // STRIDE
        img_t = self.normalize(self.to_tensor(img_r).float().contiguous())
        return {"image": img_t, "meta": {
            "orig_size": img.size, "scaled_size": img_r.size,
            "w_out": w_out, "h_out": h_out, "img_id": os.path.basename(img_path)
        }}


In [5]:
# ====== BLOK 5 — Split, Collate & DataLoader (tanpa 'density') ======
from collections import defaultdict
import os as _os

RATIOS = {"train": 0.80, "val": 0.10, "hold": 0.10}
assert abs(sum(RATIOS.values()) - 1.0) < 1e-6

all_items = list_train_items(TRAIN_IMG_DIR, TRAIN_LBL_DIR)
print(f"Total train items with labels: {len(all_items)}")

# hitung count per gambar (untuk stratified split)
counts = []
for _, lbl in tqdm(all_items, desc="Count people per image", ncols=120, bar_format="{l_bar}{bar:50}{r_bar}"):
    pts = read_points_from_json(lbl)
    counts.append(int(len(pts)))
counts = np.asarray(counts, dtype=np.int32)

bin_edges = np.array([0, 1, 11, 31, 81, 10**9])
bin_ids = np.digitize(counts, bins=bin_edges[1:-1], right=False)

indices = np.arange(len(all_items))
bins_to_indices = defaultdict(list)
for i, b in zip(indices, bin_ids):
    bins_to_indices[int(b)].append(i)

def split_bin(idx_list, ratios, seed):
    idx = np.array(idx_list, dtype=np.int32)
    if len(idx) == 0: return [], [], []
    rng = np.random.default_rng(seed); rng.shuffle(idx)
    n = len(idx)
    n_train = int(round(ratios["train"] * n))
    n_val   = int(round(ratios["val"]   * n))
    n_hold  = n - n_train - n_val
    if n >= 3:
        if n_train == 0: n_train = 1
        if n_val   == 0: n_val   = 1
        n_hold = max(1, n - n_train - n_val)
        if n_train + n_val + n_hold > n:
            reduce = (n_train + n_val + n_hold) - n
            n_train = max(1, n_train - reduce); n_hold  = n - n_train - n_val
    elif n == 2:
        n_train, n_val, n_hold = 1, 1, 0
    else:
        n_train, n_val, n_hold = 1, 0, 0
    t = idx[:n_train]; v = idx[n_train:n_train+n_val]; h = idx[n_train+n_val:n]
    return t.tolist(), v.tolist(), h.tolist()

train_idx, val_idx, hold_idx = [], [], []
rng_global = np.random.default_rng(SEED)
for b, idx_list in bins_to_indices.items():
    t, v, h = split_bin(idx_list, RATIOS, seed=SEED + int(b))
    train_idx.extend(t); val_idx.extend(v); hold_idx.extend(h)

if len(val_idx) == 0 and len(indices) > 1:
    n_pick = max(1, int(round(0.10 * len(indices))))
    pick = rng_global.choice(indices, size=n_pick, replace=False).tolist()
    val_idx.extend(pick); train_idx = [i for i in train_idx if i not in pick]

if len(hold_idx) == 0 and len(indices) > 2:
    avail = np.array([i for i in indices if i not in val_idx], dtype=np.int32)
    n_pick = max(1, int(round(0.10 * len(avail))))
    pick = rng_global.choice(avail, size=n_pick, replace=False).tolist()
    hold_idx.extend(pick); train_idx = [i for i in train_idx if i not in pick]

train_items = [all_items[i] for i in train_idx]
val_items   = [all_items[i] for i in val_idx]
hold_items  = [all_items[i] for i in hold_idx]
print(f"Split -> train: {len(train_items)} | val: {len(val_items)} | holdout: {len(hold_items)}")

train_ds = CrowdTrainDataset(train_items, train=True)
val_ds   = CrowdTrainDataset(val_items,   train=False)
hold_ds  = CrowdTrainDataset(hold_items,  train=False)

def pad_to_size(t: torch.Tensor, h: int, w: int):
    _, th, tw = t.shape
    pad_h = h - th; pad_w = w - tw
    if pad_h == 0 and pad_w == 0: return t
    return F.pad(t, (0, pad_w, 0, pad_h), mode="constant", value=0.0)

def _sanitize_points_cpu(p):
    if isinstance(p, np.ndarray):
        t = torch.from_numpy(p.astype(np.float32))
    elif isinstance(p, torch.Tensor):
        t = p.detach().to(dtype=torch.float32, device='cpu')
    else:
        return torch.zeros((0, 2), dtype=torch.float32)
    if t.numel() == 0: return torch.zeros((0, 2), dtype=torch.float32)
    if t.dim() != 2:
        if t.numel() % 2 == 0: t = t.reshape(-1, 2)
        else: return torch.zeros((0, 2), dtype=torch.float32)
    if t.shape[0] == 2 and t.shape[1] != 2: t = t.t()
    if t.shape[1] != 2: return torch.zeros((0, 2), dtype=torch.float32)
    return t.contiguous()

def batch_collate(batch):
    imgs     = [b["image"]   for b in batch]
    counts   = [b["count"]   for b in batch]
    imgpaths = [b.get("img_path", "") for b in batch]
    pts_list = [b.get("points_out_np", np.zeros((0, 2), dtype=np.float32)) for b in batch]

    Hs = [x.shape[1] for x in imgs]; Ws = [x.shape[2] for x in imgs]
    Hmax, Wmax = max(Hs), max(Ws)

    imgs_b = torch.stack([pad_to_size(x, Hmax, Wmax).contiguous() for x in imgs], dim=0)
    counts_b = torch.stack(counts, dim=0)

    imgs_b = imgs_b.to(memory_format=torch.channels_last)
    points_out = [_sanitize_points_cpu(p) for p in pts_list]
    return {"image": imgs_b, "count": counts_b,
            "img_path": imgpaths, "points_out": points_out}

PIN = (DEVICE.type == "cuda")

# Anti-hang Windows/Jupyter
if IS_WIN or IN_NOTEBOOK:
    NUM_WORKERS = 0; PERSISTENT = False; PREFETCH = None
else:
    CPU_CORES   = _os.cpu_count() or 8
    NUM_WORKERS = min(4, max(2, CPU_CORES // 4))
    PERSISTENT  = True
    PREFETCH    = 2

def _make_loader(ds, bs, shuffle):
    if PERSISTENT and PREFETCH is not None:
        return DataLoader(ds, batch_size=bs, shuffle=shuffle,
                          num_workers=NUM_WORKERS, pin_memory=PIN, drop_last=shuffle,
                          persistent_workers=True, prefetch_factor=PREFETCH, collate_fn=batch_collate)
    else:
        return DataLoader(ds, batch_size=bs, shuffle=shuffle,
                          num_workers=NUM_WORKERS, pin_memory=PIN, drop_last=shuffle,
                          collate_fn=batch_collate)

train_dl = _make_loader(train_ds, BATCH_SIZE, True)
val_dl   = _make_loader(val_ds,   1,          False)
hold_dl  = _make_loader(hold_ds,  1,          False)


Total train items with labels: 1900


Count people per image: 100%|██████████████████████████████████████████████████| 1900/1900 [00:02<00:00, 744.41it/s]

Split -> train: 1520 | val: 190 | holdout: 190


In [6]:
# ====== BLOK 6 — Model (CSR-LITE) ======
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

# Ringan tapi tetap akurat (DCN/CBAM OFF)
USE_ASPP      = True
USE_DUAL_HEAD = True
USE_GC        = True
USE_SPP       = True

EXPO_GATE_THR   = 0.6
EXPO_GATE_ALPHA = 6.0

class ASPPMini(nn.Module):
    def __init__(self, in_ch, out_ch=128, rates=(1,2,3)):
        super().__init__()
        n = len(rates); per = int(math.ceil(out_ch / n))
        self.branches = nn.ModuleList([
            nn.Sequential(nn.Conv2d(in_ch, per, 3, padding=d, dilation=d, bias=False),
                          nn.ReLU(inplace=True)) for d in rates
        ])
        self.proj = nn.Conv2d(per * n, out_ch, 1, bias=True)
    def forward(self, x):
        feats = [b(x) for b in self.branches]
        return self.proj(torch.cat(feats, dim=1))

class GCBlock(nn.Module):
    def __init__(self, ch, r=16):
        super().__init__()
        mid = max(8, ch // r)
        self.conv1 = nn.Conv2d(ch, mid, 1, bias=True)
        self.conv2 = nn.Conv2d(mid, ch, 1, bias=True)
        self.act   = nn.ReLU(inplace=True)
        self.gate  = nn.Sigmoid()
    def forward(self, x):
        c = torch.mean(x, dim=(2,3), keepdim=True)
        c = self.act(self.conv1(c)); c = self.gate(self.conv2(c))
        return x + c

class SPPRingkas(nn.Module):
    def __init__(self, in_ch, out_ch=128, bins=(1, 2, 4)):
        super().__init__()
        self.bins = bins
        self.proj = nn.Conv2d(in_ch * (1 + len(bins)), out_ch, 1, bias=True)
    def forward(self, x):
        B, C, H, W = x.shape
        feats = [x]
        for b in self.bins:
            p = F.adaptive_avg_pool2d(x, (b, b))
            p = F.interpolate(p, size=(H, W), mode="bilinear", align_corners=False)
            feats.append(p)
        return self.proj(torch.cat(feats, dim=1))

class EfficientB0_Frontend(nn.Module):
    def __init__(self, target_stride: int = 16, freeze_all: bool = False):
        super().__init__()
        assert target_stride in (4, 8, 16)
        self.target_stride = target_stride
        try:
            base = efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)
        except Exception:
            base = efficientnet_b0(weights=None)
        feats = list(base.features.children())
        dummy = torch.zeros(1, 3, 256, 256)
        layers = []; H0 = dummy.shape[2]; cur = dummy; stop_idx = -1
        with torch.no_grad():
            for i, m in enumerate(feats):
                cur = m(cur); down = H0 // cur.shape[2]
                layers.append(m)
                if down == self.target_stride: stop_idx = i; break
                if down >  self.target_stride: layers.pop(); stop_idx = i-1; break
        if stop_idx < 0: layers = feats
        self.stem = nn.Sequential(*layers)
        with torch.no_grad(): out = self.stem(dummy)
        self.out_channels = out.shape[1]
        for m in self.stem.modules():
            if isinstance(m, nn.BatchNorm2d): m.eval()
        if freeze_all:
            for p in self.stem.parameters(): p.requires_grad = False
        print(f"[EffB0] stride={self.target_stride} | stem={len(self.stem)} | C_out={self.out_channels} | frozen={freeze_all}")

    def forward(self, x): return self.stem(x)

class CSRLike_Backend_Lite(nn.Module):
    """CSR-like backend dengan channel lebih kecil: 192/384/128."""
    def __init__(self, in_ch: int):
        super().__init__()
        def C(in_c, out_c, d=2):
            pad = d
            return nn.Sequential(nn.Conv2d(in_c, out_c, 3, padding=pad, dilation=d, bias=True),
                                 nn.ReLU(inplace=True))
        # from: [256,256,256, 512,512,512, 256,128]
        # to  : [192,192,192, 384,384,384, 192,128]
        self.layers = nn.ModuleList([
            C(in_ch, 192, d=2),  # 0
            C(192,   192, d=2),  # 1
            C(192,   192, d=2),  # 2
            C(192,   384, d=2),  # 3
            C(384,   384, d=2),  # 4
            C(384,   384, d=2),  # 5
            C(384,   192, d=2),  # 6
            C(192,   128, d=2),  # 7
        ])
        self.aspp = ASPPMini(128, out_ch=128, rates=(1,2,3)) if USE_ASPP else nn.Identity()
        self.gc   = GCBlock(128)        if USE_GC  else nn.Identity()
        self.spp  = SPPRingkas(128,128) if USE_SPP else nn.Identity()
        self.head_norm   = nn.Conv2d(128, 1, 1)
        self.head_bright = nn.Conv2d(128, 1, 1) if USE_DUAL_HEAD else None
        self.softplus    = nn.Softplus()
        nn.init.constant_(self.head_norm.bias, 0.1)
        if self.head_bright is not None: nn.init.constant_(self.head_bright.bias, 0.1)

    def forward(self, x, expo_gate: torch.Tensor | None = None):
        x = self.layers[0](x); x = self.layers[1](x); x = self.layers[2](x)
        x = self.layers[3](x); x = self.layers[4](x); x = self.layers[5](x)
        x = self.layers[6](x); x = self.layers[7](x)
        x = self.aspp(x); x = self.gc(x); x = self.spp(x)
        if USE_DUAL_HEAD and (expo_gate is not None):
            y_n = self.head_norm(x); y_b = self.head_bright(x)
            g = expo_gate.view(-1, 1, 1, 1); y = (1.0 - g) * y_n + g * y_b
        else:
            y = self.head_norm(x)
        return self.softplus(y)

class EfficientCSRNet(nn.Module):
    def __init__(self, target_stride: int = STRIDE):
        super().__init__()
        self.front = EfficientB0_Frontend(target_stride=target_stride, freeze_all=False)
        self.back  = CSRLike_Backend_Lite(self.front.out_channels)
        self.register_buffer("_mean", torch.tensor([0.485, 0.456, 0.406]).view(1,3,1,1))
        self.register_buffer("_std",  torch.tensor([0.229, 0.224, 0.225]).view(1,3,1,1))
        print(f"[Back] CSR-LITE | ASPP={USE_ASPP} | GC={USE_GC} | SPP={USE_SPP} | DualHead={USE_DUAL_HEAD}")

    @torch.no_grad()
    def _exposure_gate(self, x_norm: torch.Tensor) -> torch.Tensor:
        x = torch.clamp(x_norm * self._std + self._mean, 0.0, 1.0)
        Y = 0.299*x[:,0:1] + 0.587*x[:,1:2] + 0.114*x[:,2:3]
        y_mean = Y.mean(dim=(1,2,3))
        return torch.sigmoid(EXPO_GATE_ALPHA * (y_mean - EXPO_GATE_THR))

    def forward(self, x):
        f = self.front(x)
        if USE_DUAL_HEAD:
            g = self._exposure_gate(x)
            return self.back(f, expo_gate=g)
        else:
            return self.back(f, expo_gate=None)

model = EfficientCSRNet(target_stride=STRIDE).to(DEVICE)
model = model.to(memory_format=torch.channels_last)
if USE_TORCH_COMPILE:
    try:
        model = torch.compile(model, mode="max-autotune", fullgraph=False)
        print("[compile] torch.compile enabled")
    except Exception as e:
        print(f"[compile] skipped: {e}")
print("Model params (M):", sum(p.numel() for p in model.parameters())/1e6)


[EffB0] stride=16 | stem=5 | C_out=80 | frozen=False
[Back] CSR-LITE | ASPP=True | GC=True | SPP=True | DualHead=True
Model params (M): 5.54835


In [7]:
# ====== BLOK 7 — Loss utils (dtype-aware Gaussian; ACCUM-aware OneCycle) ======
crit_density = nn.SmoothL1Loss(beta=0.01)
crit_count   = nn.L1Loss()

_FAST_GAUSS_CACHE = {}
def _gauss2d_kernel(sigma_pix: float, device, dtype):
    """
    Kernel Gaussian dengan dtype yang SAMA dengan prediksi (fp16/fp32)
    + cache per (sigma, device, dtype) untuk hemat alokasi.
    """
    key = (int(round(float(sigma_pix) * 100)), str(device), str(dtype))
    if key in _FAST_GAUSS_CACHE:
        return _FAST_GAUSS_CACHE[key]
    k = max(3, int(6 * float(sigma_pix)) | 1)
    r = k // 2
    ys = torch.arange(-r, r + 1, device=device, dtype=torch.float32)
    xs = torch.arange(-r, r + 1, device=device, dtype=torch.float32)
    Y, X = torch.meshgrid(ys, xs, indexing="ij")
    g = torch.exp(-(X * X + Y * Y) / (2.0 * (sigma_pix ** 2) + 1e-6))
    g = g / (g.sum() + 1e-8)
    weight = g.to(dtype=dtype).view(1, 1, k, k).contiguous()
    _FAST_GAUSS_CACHE[key] = weight
    return weight

def _impulse_from_points(batch_points, H, W, device, dtype):
    """Bangun impulse map (B,1,H,W) di GPU dengan dtype yang cocok (fp16/fp32)."""
    B = len(batch_points)
    imp = torch.zeros((B, 1, H, W), device=device, dtype=dtype)
    for b, pts in enumerate(batch_points):
        if pts is None or (not torch.is_tensor(pts)) or pts.numel() == 0: continue
        pts = pts.detach().to(device=device, dtype=torch.float32)
        if pts.dim() != 2:
            if pts.numel() % 2 == 0: pts = pts.view(-1, 2)
            else: continue
        if pts.shape[0] == 2 and pts.shape[1] != 2: pts = pts.t()
        if pts.shape[1] != 2: continue
        x = pts[:, 0].round().clamp_(0, W - 1).to(torch.long)
        y = pts[:, 1].round().clamp_(0, H - 1).to(torch.long)
        lin = y * W + x
        imp[b, 0].view(-1).scatter_add_(0, lin, torch.ones_like(lin, dtype=imp.dtype, device=device))
    return imp

def _normalize_prob_map(T, eps=1e-8):
    S = T.sum(dim=(2, 3), keepdim=True)
    mask_zero = (S <= 0) | (~torch.isfinite(S))
    if mask_zero.any():
        B, _, H, W = T.shape
        T[mask_zero] = 1.0 / (H * W)
        S = T.sum(dim=(2, 3), keepdim=True)
    return (T / (S + eps)).clamp_min(eps)

def kl_from_blur_target(pred_den: torch.Tensor, blur_target: torch.Tensor) -> torch.Tensor:
    """KL(T||P) dengan T hasil blur (detach, dtype sudah cocok)."""
    B, _, H, W = pred_den.shape
    T = _normalize_prob_map(blur_target)
    S_pred = pred_den.sum(dim=(2, 3), keepdim=True)
    P = torch.where((S_pred > 0) & torch.isfinite(S_pred),
                    pred_den / (S_pred + 1e-8),
                    torch.full_like(pred_den, 1.0 / (H * W)))
    P = P.clamp_min(1e-8)
    logP = P.log()
    return F.kl_div(logP, T, reduction="sum") / B

optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scaler    = torch.cuda.amp.GradScaler(enabled=(DEVICE.type == "cuda"))

def current_lr(optim): return optim.param_groups[0]["lr"]

def loss_weights_for_epoch(epoch: int):
    align_w = OT_WEIGHT_WARM if epoch <= 2 else OT_WEIGHT_BASE
    count_w = COUNT_WEIGHT_WARM if epoch <= 2 else COUNT_WEIGHT_BASE
    return align_w, count_w, LOCAL_WEIGHT, DENSITY_ALPHA

# ACCUM-aware OneCycle: hitung berdasarkan JUMLAH optimizer.step()
_steps_per_epoch_loader = max(1, len(train_dl))
OPT_STEPS_PER_EPOCH     = max(1, math.ceil(_steps_per_epoch_loader / ACCUM_STEPS))
TOTAL_OPT_STEPS         = EPOCHS * OPT_STEPS_PER_EPOCH
MAX_LR = 6e-4
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=MAX_LR, total_steps=TOTAL_OPT_STEPS,
    pct_start=0.15, div_factor=10.0, final_div_factor=100.0,
    anneal_strategy="cos", three_phase=False
)


In [8]:
# ====== BLOK 8 — Train & Val (OOM-safe: no_grad target + micro-chunk) ======
def _bar(desc):
    return dict(desc=desc, ncols=120, bar_format="{l_bar}{bar:50}{r_bar}")

def batch_count(t: torch.Tensor) -> torch.Tensor:
    return t.flatten(1).sum(dim=1)

def avgpool_local_sum(t: torch.Tensor, k: int = 8):
    return F.avg_pool2d(t, kernel_size=k, stride=k, padding=0) * (k * k)

def _forward_loss_chunk(img_chunk, pts_list_chunk, gauss_w, gauss_pad):
    """Forward + loss untuk satu chunk kecil (hemat VRAM)."""
    with torch.amp.autocast(device_type=("cuda" if DEVICE.type == "cuda" else "cpu"),
                            enabled=(DEVICE.type == "cuda")):
        pred = model(img_chunk)                       # (b,1,H,W) fp16/fp32
        Bc, _, H, W = pred.shape
        # --- bangun target blur TANPA grad & SAMA dtype ---
        with torch.no_grad():
            imp = _impulse_from_points(pts_list_chunk, H, W, device=pred.device, dtype=pred.dtype)  # (b,1,H,W)
            blur_gt = F.conv2d(imp, gauss_w, padding=gauss_pad)                                     # (b,1,H,W)
        # 1) KL(T||P)
        align = kl_from_blur_target(pred, blur_gt)
        # 2) Global count
        pred_cnt = batch_count(pred)
        gt_cnt   = batch_count(imp)
        lc  = crit_count(pred_cnt, gt_cnt)
        # 3) Local block-sum
        k = 6
        pred_loc = avgpool_local_sum(pred, k=k)
        gt_loc   = avgpool_local_sum(blur_gt,  k=k)
        lloc = crit_count(pred_loc, gt_loc)
        # 4) Stabilizer
        ld = crit_density(pred, blur_gt)
        return align, lc, lloc, ld, pred_cnt.mean().detach(), gt_cnt.mean().detach()

def train_one_epoch(model, loader, optimizer, scaler, epoch: int, num_epochs: int, scheduler=None):
    model.train()
    running = running_align = running_cc = running_ld = running_loc = 0.0
    n = 0
    pbar = tqdm(loader, **_bar(f"Train {epoch}/{num_epochs}"))

    align_w, count_w, local_w, dens_w = loss_weights_for_epoch(epoch)
    optimizer.zero_grad(set_to_none=True)

    # kernel Gaussian sesuai dtype pred (ditentukan saat chunk pertama)
    gauss_w = None; gauss_pad = None

    accum_counter = 0  # jumlah DataLoader steps yang terakumulasi
    for batch in pbar:
        img_full = batch["image"].to(DEVICE, non_blocking=True).to(memory_format=torch.channels_last)
        pts_full = [t.to(DEVICE, non_blocking=True, dtype=torch.float32) for t in batch["points_out"]]

        B = img_full.size(0)
        CHUNK = min(MICRO_CHUNK, B)
        num_chunks = (B + CHUNK - 1) // CHUNK

        # tentukan dtype pred via warmup 1 kali (pakai fp16 saat autocast)
        if gauss_w is None:
            with torch.amp.autocast(device_type=("cuda" if DEVICE.type == "cuda" else "cpu"),
                                    enabled=(DEVICE.type == "cuda")):
                _pred_warm = model(img_full[:1])
            gauss_w = _gauss2d_kernel(2.0, device=_pred_warm.device, dtype=_pred_warm.dtype)
            gauss_pad = gauss_w.shape[-1] // 2
            del _pred_warm

        # akumulasi loss dari semua chunk (dibagi num_chunks)
        loss_sum_chunks = 0.0
        pc_show = gc_show = 0.0

        for s in range(num_chunks):
            lo, hi = s*CHUNK, min((s+1)*CHUNK, B)
            img = img_full[lo:hi]
            pts = pts_full[lo:hi]

            align, lc, lloc, ld, pc_m, gc_m = _forward_loss_chunk(img, pts, gauss_w, gauss_pad)
            loss = (align_w*align + count_w*lc + local_w*lloc + dens_w*ld) / float(num_chunks)  # bagi per-chunk
            loss = loss / ACCUM_STEPS  # grad-accum

            scaler.scale(loss).backward()

            loss_sum_chunks += loss.item() * ACCUM_STEPS  # kembalikan skala untuk logging
            pc_show = pc_m.item(); gc_show = gc_m.item()

        # selesai 1 step DataLoader
        accum_counter += 1
        if accum_counter % ACCUM_STEPS == 0:
            scaler.unscale_(optimizer)
            nn.utils.clip_grad_norm_(model.parameters(), CLIP_GRAD)
            scaler.step(optimizer); scaler.update()
            optimizer.zero_grad(set_to_none=True)
            if scheduler is not None: scheduler.step()

        # logging (pakai nilai terakhir per-batch)
        bs = B
        running       += loss_sum_chunks * bs; n += bs
        running_align += align.item() * bs
        running_cc    += lc.item() * bs
        running_loc   += lloc.item() * bs
        running_ld    += ld.item() * bs

        pbar.set_postfix(
            loss=f"{(running/max(1,n)):.4f}",
            align=f"{(running_align/max(1,n)):.4f}",
            cnt=f"{(running_cc/max(1,n)):.2f}",
            loc=f"{(running_loc/max(1,n)):.3f}",
            ld=f"{(running_ld/max(1,n)):.4f}",
            lr=f"{current_lr(optimizer):.2e}",
            pc=f"{pc_show:.1f}", gc=f"{gc_show:.1f}"
        )
    pbar.close()
    return (running/max(1,n)), (running_align/max(1,n)), (running_cc/max(1,n)), (running_loc/max(1,n)), (running_ld/max(1,n))

@torch.no_grad()
def evaluate_count_mae_rmse(model, loader, desc="Val"):
    model.eval()
    mae, mse = 0.0, 0.0
    pbar = tqdm(loader, **_bar(desc))
    for batch in pbar:
        img = batch["image"].to(DEVICE, non_blocking=True).to(memory_format=torch.channels_last)
        gt_count = float(batch["count"].item())
        with torch.amp.autocast(device_type=("cuda" if DEVICE.type == "cuda" else "cpu"),
                                enabled=(DEVICE.type == "cuda")):
            pred_den = model(img)
        pred_count = float(pred_den.detach().sum().item())
        err = abs(pred_count - gt_count)
        mae += err; mse += err * err
        pbar.set_postfix(last_err=f"{err:.2f}", pc=f"{pred_count:.1f}", gc=f"{gt_count:.1f}")
    pbar.close()
    N = max(1, len(loader))
    return mae / N, math.sqrt(mse / N)


In [ ]:
# ====== BLOK 9 — Loop Training (RUN_VAL_EVERY) ======
def _quick_count_stats(items, title):
    cnts = []
    for _, lbl in items:
        pts = read_points_from_json(lbl)
        cnts.append(len(pts))
    if len(cnts) == 0:
        print(f"[{title}] kosong")
    else:
        a = np.array(cnts)
        print(f"[{title}] n={len(a)} | mean={a.mean():.1f} | median={np.median(a):.1f} | max={a.max()} | min={a.min()}")

_quick_count_stats(train_items, "TRAIN")
_quick_count_stats(val_items,   "VAL")
_quick_count_stats(hold_items,  "HOLD")

best_mae = 1e9
best_path = os.path.join(OUT_DIR, "best_model.pth")
last_path = os.path.join(OUT_DIR, "last_model.pth")

with open(LOG_CSV, "w", newline="") as f:
    w = csv.writer(f)
    w.writerow([
        "epoch", "train_loss", "train_align", "train_count",
        "train_local", "train_stab", "val_mae", "val_rmse",
        "lr", "time_s"
    ])

RUN_VAL_EVERY = 1  # validasi tiap 3 epoch (kecuali 5 epoch terakhir per-epoch)

EPOCHS_RUN = EPOCHS
for epoch in range(1, EPOCHS_RUN + 1):
    t0 = time.time()
    tr_loss, tr_align, tr_c, tr_loc, tr_ld = train_one_epoch(
        model, train_dl, optimizer, scaler, epoch, EPOCHS_RUN, scheduler
    )

    do_val = (epoch % RUN_VAL_EVERY == 0) or (epoch > EPOCHS_RUN - 5)
    if do_val:
        val_mae, val_rmse = evaluate_count_mae_rmse(model, val_dl, desc="Val")
    else:
        val_mae, val_rmse = float("nan"), float("nan")

    elapsed = time.time() - t0
    print(f"[{epoch:02d}/{EPOCHS_RUN}] loss={tr_loss:.4f} (align={tr_align:.4f}, c={tr_c:.2f}, "
          f"loc={tr_loc:.3f}, stab={tr_ld:.4f}) | VAL: MAE={val_mae:.3f} RMSE={val_rmse:.3f} "
          f"| lr={current_lr(optimizer):.2e} | {elapsed:.1f}s")

    with open(LOG_CSV, "a", newline="") as f:
        w = csv.writer(f)
        w.writerow([
            epoch, f"{tr_loss:.6f}", f"{tr_align:.6f}", f"{tr_c:.6f}",
            f"{tr_loc:.6f}", f"{tr_ld:.6f}", f"{val_mae:.6f}",
            f"{val_rmse:.6f}", f"{current_lr(optimizer):.6e}", f"{elapsed:.2f}"
        ])

    if do_val and (val_mae < best_mae):
        best_mae = val_mae
        torch.save({"model": model.state_dict(), "epoch": epoch,
                    "val_mae": val_mae, "val_rmse": val_rmse},
                   best_path)
        print(f"  -> Saved BEST (VAL MAE {best_mae:.3f}) to {best_path}")

torch.save({"model": model.state_dict(), "epoch": EPOCHS_RUN}, last_path)
print("Saved LAST to", last_path)


[TRAIN] n=1520 | mean=136.9 | median=28.0 | max=2673 | min=1
[VAL] n=190 | mean=147.6 | median=29.5 | max=3139 | min=2
[HOLD] n=190 | mean=146.4 | median=27.5 | max=2261 | min=2


Train 1/20: 100%|██████████████████████████████████████████████████| 190/190 [08:07<00:00,  2.57s/it, align=1.1284, cnt=


[01/20] loss=5.3384 (align=1.1284, c=191.55, loc=4.413, stab=0.1159) | VAL: MAE=nan RMSE=nan | lr=1.96e-04 | 487.5s


Train 2/20: 100%|██████████████████████████████████████████████████| 190/190 [07:44<00:00,  2.44s/it, align=0.6169, cnt=


[02/20] loss=2.6715 (align=0.6169, c=88.43, loc=2.438, stab=0.0616) | VAL: MAE=nan RMSE=nan | lr=4.67e-04 | 464.3s


...

[18/20] loss=0.6050 (align=0.1319, c=23.40, loc=0.875, stab=0.0230) | VAL: MAE=32.836 RMSE=82.162 | lr=2.06e-05 | 462.1s


Train 19/20: 100%|██████████████████████████████████████████████████| 190/190 [07:36<00:00,  2.40s/it, align=0.1224, cnt
Val: 100%|██████████████████████████████████████████████████| 190/190 [00:05<00:00, 32.32it/s, gc=4.0, last_err=0.62, pc


[19/20] loss=0.5714 (align=0.1224, c=18.85, loc=0.782, stab=0.0208) | VAL: MAE=35.683 RMSE=104.232 | lr=5.60e-06 | 462.5s


Train 20/20: 100%|██████████████████████████████████████████████████| 190/190 [07:38<00:00,  2.41s/it, align=0.1182, cnt
Val: 100%|██████████████████████████████████████████████████| 190/190 [00:05<00:00, 32.52it/s, gc=4.0, last_err=0.62, pc


[20/20] loss=0.5493 (align=0.1182, c=17.31, loc=0.744, stab=0.0199) | VAL: MAE=38.737 RMSE=110.479 | lr=6.01e-07 | 464.0s
Saved LAST to C:\Python\Template\Data Science\test\HOLOGY_UB_2025\out_csrnet_efficientb0\last_model.pth


In [9]:
# ====== BLOK 10 (robust) ======
# Pastikan _bar, evaluate_count_mae_rmse, hold_dl, model, DEVICE, OUT_DIR sudah ada dari blok sebelumnya

USE_WHICH = "last"   # "best" atau "last"
CAND_BEST = os.path.join(OUT_DIR, "best_model.pth")
CAND_LAST = os.path.join(OUT_DIR, "last_model.pth")

# Pilih model path dengan fallback yang jelas
if USE_WHICH == "best" and os.path.exists(CAND_BEST):
    MODEL_PATH = CAND_BEST
elif USE_WHICH == "last" and os.path.exists(CAND_LAST):
    MODEL_PATH = CAND_LAST
elif os.path.exists(CAND_BEST):
    print("[WARN] USE_WHICH=last tapi last tidak ada. Fallback ke BEST.")
    MODEL_PATH = CAND_BEST
elif os.path.exists(CAND_LAST):
    print("[WARN] USE_WHICH=best tapi best tidak ada. Fallback ke LAST.")
    MODEL_PATH = CAND_LAST
else:
    raise FileNotFoundError(f"Tidak menemukan checkpoint di {CAND_BEST} maupun {CAND_LAST}.")

ckpt = torch.load(MODEL_PATH, map_location=DEVICE)
sd = ckpt["model"] if isinstance(ckpt, dict) and "model" in ckpt else ckpt
model.load_state_dict(sd); model.eval()
print(f"[HOLDOUT] Loading: {MODEL_PATH}")
print(f"  epoch={ckpt.get('epoch', None)}, val_mae={ckpt.get('val_mae', None)}")

EVAL_PRED_OFFSET = 0
EVAL_MIN_COUNT   = 1

TTA_SCALES = [1.0]  # set ke [0.8,1.0,1.2] jika ingin TTA
def infer_count_tta(img_tensor: torch.Tensor) -> float:
    total = 0.0
    with torch.no_grad(), torch.amp.autocast(device_type=("cuda" if DEVICE.type=="cuda" else "cpu"),
                                             enabled=(DEVICE.type == "cuda")):
        for s in TTA_SCALES:
            if s == 1.0:
                pred = model(img_tensor.to(DEVICE, non_blocking=True).to(memory_format=torch.channels_last))
            else:
                _, C, H, W = img_tensor.shape
                Hs = int(round(H * s)); Ws = int(round(W * s))
                img_s = F.interpolate(img_tensor, size=(Hs, Ws), mode="bilinear", align_corners=False)
                pred = model(img_s.to(DEVICE, non_blocking=True).to(memory_format=torch.channels_last))
            total += float(pred.detach().sum().item())
    return total / len(TTA_SCALES)

def postprocess_normal(count_raw: float, offset: float = 0.0, min_count: int = 1) -> int:
    c = math.ceil(count_raw + float(offset))
    return max(int(c), int(min_count))

def postprocess_piecewise(count_raw: float, offset: float = 0.0, min_count: int = 1) -> int:
    c = count_raw + float(offset)
    c = int(round(c)) if c < 50 else int(math.ceil(c))
    return max(int(c), int(min_count))

rows_full = []
abs_ntta_raw, abs_tta_raw = [], []
abs_ntta_norm, abs_ntta_piece = [], []
abs_tta_norm,  abs_tta_piece  = [], []

pbar = tqdm(hold_dl, **_bar("Holdout eval: RAW + (non-TTA vs TTA) + rounding"))
for batch in pbar:
    img_cpu = batch["image"]
    img = img_cpu.to(DEVICE, non_blocking=True).to(memory_format=torch.channels_last)
    gt_count = float(batch["count"].item())

    with torch.amp.autocast(device_type=("cuda" if DEVICE.type=="cuda" else "cpu"),
                            enabled=(DEVICE.type == "cuda")):
        pred_den = model(img)
    raw_ntta = float(pred_den.detach().sum().item())
    raw_tta = infer_count_tta(img_cpu)

    ae_ntta_raw = abs(raw_ntta - gt_count); abs_ntta_raw.append(ae_ntta_raw)
    ae_tta_raw  = abs(raw_tta  - gt_count); abs_tta_raw.append(ae_tta_raw)

    ntta_norm  = postprocess_normal(raw_ntta,  EVAL_PRED_OFFSET, EVAL_MIN_COUNT)
    ntta_piece = postprocess_piecewise(raw_ntta, EVAL_PRED_OFFSET, EVAL_MIN_COUNT)
    tta_norm   = postprocess_normal(raw_tta,   EVAL_PRED_OFFSET, EVAL_MIN_COUNT)
    tta_piece  = postprocess_piecewise(raw_tta,  EVAL_PRED_OFFSET, EVAL_MIN_COUNT)

    abs_ntta_norm.append(abs(ntta_norm  - gt_count))
    abs_ntta_piece.append(abs(ntta_piece - gt_count))
    abs_tta_norm.append(abs(tta_norm   - gt_count))
    abs_tta_piece.append(abs(tta_piece - gt_count))

    img_id = batch.get("img_path", ["unknown.jpg"])
    img_id = img_id[0] if isinstance(img_id, list) else img_id
    img_id = os.path.basename(img_id)

    rows_full.append((img_id, gt_count, raw_ntta, raw_tta,
                      ntta_norm, ntta_piece, tta_norm, tta_piece))
    pbar.set_postfix(AE_RAW_nTTA=f"{ae_ntta_raw:.1f}", AE_RAW_TTA=f"{ae_tta_raw:.1f}", gt=f"{gt_count:.1f}")
pbar.close()

def mae_rmse(abs_list):
    mae = float(np.mean(abs_list)) if len(abs_list) else float("nan")
    rmse = float(np.sqrt(np.mean(np.square(abs_list)))) if len(abs_list) else float("nan")
    return mae, rmse

N = max(1, len(rows_full))
mae_raw_nTTA, rmse_raw_nTTA = mae_rmse(abs_ntta_raw)
mae_raw_TTA,  rmse_raw_TTA  = mae_rmse(abs_tta_raw)
mae_nN, rmse_nN = mae_rmse(abs_ntta_norm)
mae_nP, rmse_nP = mae_rmse(abs_ntta_piece)
mae_tN, rmse_tN = mae_rmse(abs_tta_norm)
mae_tP, rmse_tP = mae_rmse(abs_tta_piece)

print(f"[HOLDOUT][RAW]     NON-TTA → MAE={mae_raw_nTTA:.3f} | RMSE={rmse_raw_nTTA:.3f} | n={N}")
print(f"[HOLDOUT][RAW]     TTA{TTA_SCALES} → MAE={mae_raw_TTA:.3f} | RMSE={rmse_raw_TTA:.3f} | n={N}")
print(f"[HOLDOUT][Rounded] NON-TTA (ceil) → MAE={mae_nN:.3f} | RMSE={rmse_nN:.3f} | n={N}")
print(f"[HOLDOUT][Rounded] NON-TTA (piecewise) → MAE={mae_nP:.3f} | RMSE={rmse_nP:.3f} | n={N}")
print(f"[HOLDOUT][Rounded] TTA (ceil) → MAE={mae_tN:.3f} | RMSE={rmse_tN:.3f} | n={N}")
print(f"[HOLDOUT][Rounded] TTA (piecewise) → MAE={mae_tP:.3f} | RMSE={rmse_tP:.3f} | n={N}")

# (Ringkasan per-bin prediksi Non-TTA ceil)
def in_bin(v, lo, hi, left_inclusive=True):
    if hi is None: return (v > lo) if not left_inclusive else (v >= lo)
    return ((v >= lo) if left_inclusive else (v > lo)) and (v <= hi)

def safe_mean(a): return float(np.mean(a)) if len(a) > 0 else float("nan")

rows_ntta_norm = [(rid, gt, ntta_norm, (ntta_norm - gt))
                  for (rid, gt, _rawN, _rawT, ntta_norm, _ntta_piece, _tta_norm, _tta_piece) in rows_full]

bins = [("1-20", 1, 20, True), ("21-50", 21, 50, True), ("50-100", 50, 100, False),
        ("100-500", 100, 500, False), (">500", 500, None, False)]

summary_rows = []
print("\n[HOLDOUT] Rincian per horizon (berdasarkan pred NON-TTA ceil):")
for name, lo, hi, left_inc in bins:
    diffs = [diff for (_id, gt, pred, diff) in rows_ntta_norm if in_bin(pred, lo, hi, left_inc)]
    n_bin = len(diffs)
    if n_bin == 0:
        print(f"  {name:>7}: n=0")
        summary_rows.append([name, 0, float("nan"), float("nan"),
                             0, 0.0, float("nan"), 0, 0.0, float("nan"), 0, 0.0, float("nan")])
        continue
    diffs = np.array(diffs, dtype=np.float32)
    mae_bin   = safe_mean(np.abs(diffs))
    mse_bin   = safe_mean(diffs**2)
    rmse_bin  = float(np.sqrt(mse_bin)) if np.isfinite(mse_bin) else float("nan")
    bias_bin  = safe_mean(diffs)
    mask_pos = diffs > 0; mask_neg = diffs < 0; mask_zero= diffs == 0
    n_pos = int(mask_pos.sum()); n_neg = int(mask_neg.sum()); n_zero = int(mask_zero.sum())
    pos_ratio = n_pos / n_bin; neg_ratio = n_neg / n_bin; zero_ratio = n_zero / n_bin
    mean_pos_mag = safe_mean(diffs[mask_pos]); mean_neg_mag = safe_mean(-diffs[mask_neg])

    print(f"  {name:>7}: n={n_bin:4d} | MAE={mae_bin:7.3f} | RMSE={rmse_bin:7.3f} | Bias={bias_bin:7.3f} | "
          f"Over: n={n_pos:4d} ({pos_ratio:5.1%}), mean+={mean_pos_mag:7.3f} | "
          f"Under: n={n_neg:4d} ({neg_ratio:5.1%}), mean-={mean_neg_mag:7.3f} | "
          f"Equal: n={n_zero:4d} ({zero_ratio:5.1%})")

    summary_rows.append([name, n_bin, mae_bin, rmse_bin,
                         n_pos, pos_ratio, mean_pos_mag,
                         n_neg, neg_ratio, mean_neg_mag,
                         n_zero, zero_ratio, bias_bin])

df_samples = pd.DataFrame(rows_full, columns=[
    "image_id","gt_count","pred_raw_non_tta","pred_raw_tta",
    "pred_non_tta_normal","pred_non_tta_piecewise","pred_tta_normal","pred_tta_piecewise"
])
df_samples["diff_raw_non_tta"]       = df_samples["pred_raw_non_tta"] - df_samples["gt_count"]
df_samples["diff_raw_tta"]           = df_samples["pred_raw_tta"]     - df_samples["gt_count"]
df_samples["diff_non_tta_normal"]    = df_samples["pred_non_tta_normal"]    - df_samples["gt_count"]
df_samples["diff_non_tta_piecewise"] = df_samples["pred_non_tta_piecewise"] - df_samples["gt_count"]
df_samples["diff_tta_normal"]        = df_samples["pred_tta_normal"]        - df_samples["gt_count"]
df_samples["diff_tta_piecewise"]     = df_samples["pred_tta_piecewise"]     - df_samples["gt_count"]

df_summary = pd.DataFrame(summary_rows, columns=[
    "horizon","n","mae","rmse","n_over","ratio_over","mean_over_mag",
    "n_under","ratio_under","mean_under_mag","n_equal","ratio_equal","bias_signed"
])

xlsx_path   = os.path.join(OUT_DIR, "holdout_eval.xlsx")
csv_samples = os.path.join(OUT_DIR, "holdout_eval_samples.csv")
csv_summary = os.path.join(OUT_DIR, "holdout_eval_summary.csv")

try:
    with pd.ExcelWriter(xlsx_path, engine="openpyxl") as writer:
        df_samples.to_excel(writer, index=False, sheet_name="samples_full")
        df_summary.to_excel(writer, index=False, sheet_name="summary_non_tta_normal_bins")
    print(f"[HOLDOUT] Disimpan ke Excel: {xlsx_path}")
except Exception as e:
    print(f"[WARN] Gagal simpan .xlsx ({e}). Simpan CSV.")
    df_samples.to_csv(csv_samples, index=False)
    df_summary.to_csv(csv_summary, index=False)
    print(f"[HOLDOUT] CSV: {csv_samples}")
    print(f"[HOLDOUT] CSV: {csv_summary}")


[HOLDOUT] Loading: C:\Python\Template\Data Science\test\HOLOGY_UB_2025\out_csrnet_efficientb0\last_model.pth
  epoch=20, val_mae=None


Holdout eval: RAW + (non-TTA vs TTA) + rounding: 100%|██████████████████████████████████████████████████| 190/190 [00:35

[HOLDOUT][RAW]     NON-TTA → MAE=23.778 | RMSE=61.887 | n=190
[HOLDOUT][RAW]     TTA[1.0] → MAE=23.778 | RMSE=61.887 | n=190
[HOLDOUT][Rounded] NON-TTA (ceil) → MAE=23.942 | RMSE=61.923 | n=190
[HOLDOUT][Rounded] NON-TTA (piecewise) → MAE=23.837 | RMSE=61.919 | n=190
[HOLDOUT][Rounded] TTA (ceil) → MAE=23.942 | RMSE=61.923 | n=190
[HOLDOUT][Rounded] TTA (piecewise) → MAE=23.837 | RMSE=61.919 | n=190

[HOLDOUT] Rincian per horizon (berdasarkan pred NON-TTA ceil):
     1-20: n=  76 | MAE=  2.368 | RMSE=  3.645 | Bias=  1.132 | Over: n=  45 (59.2%), mean+=  2.956 | Under: n=  18 (23.7%), mean-=  2.611 | Equal: n=  13 (17.1%)
    21-50: n=  34 | MAE=  6.912 | RMSE=  8.607 | Bias=  3.029 | Over: n=  25 (73.5%), mean+=  6.760 | Under: n=   7 (20.6%), mean-=  9.429 | Equal: n=   2 ( 5.9%)
   50-100: n=  16 | MAE= 15.250 | RMSE= 22.068 | Bias=  6.125 | Over: n=  10 (62.5%), mean+= 17.100 | Under: n=   6 (37.5%), mean-= 12.167 | Equal: n=   0 ( 0.0%)
  100-500: n=  54 | MAE= 36.796 | RMSE= 55.4

In [10]:
# ====== BLOK 11 — Retrain 100% + SWA (Windows/Notebook safe) ======
import gc
from torch.optim.swa_utils import AveragedModel, SWALR, update_bn

RETRAIN_ENABLE    = True
RETRAIN_EPOCHS    = 4            # sesuaikan waktu
RETRAIN_LR_SCALE  = 0.5
SWA_EPOCHS        = 4            # SWA semua epoch juga OK; bisa 2–4
FT_PATIENCE       = 3            # early-stop berdasarkan holdout

best_full_path = None
best_full_path_swa = None

def eval_holdout_mae(model):
    # gunakan evaluator yang sudah ada
    mae, rmse = evaluate_count_mae_rmse(model, hold_dl, desc="Hold(FT)")
    return mae, rmse

if RETRAIN_ENABLE:
    print("\n[RETRAIN] Fine-tuning on train+val with small LR + SWA + holdout early-stop ...")
    # Bersihkan memori supaya tidak fragmentasi saat inisialisasi DataLoader
    gc.collect()
    if DEVICE.type == "cuda":
        torch.cuda.empty_cache()

    # Mulai dari BEST jika ada, else LAST
    start_ckpt_path = CAND_BEST if os.path.exists(CAND_BEST) else (CAND_LAST if os.path.exists(CAND_LAST) else None)
    if start_ckpt_path is None:
        raise FileNotFoundError("Tidak menemukan best/last untuk mulai fine-tune. Jalankan training utama dulu.")
    ckpt = torch.load(start_ckpt_path, map_location=DEVICE)
    sd = ckpt["model"] if isinstance(ckpt, dict) and "model" in ckpt else ckpt
    model.load_state_dict(sd); model.train()
    print(f"[RETRAIN] Start from: {start_ckpt_path}")

    # ---------- DataLoader retrain: single-process (anti-hang Windows/Notebook) ----------
    def make_loader_for_fulltrain(train_items, val_items, batch_size=BATCH_SIZE):
        merged_items = train_items + val_items
        ds = CrowdTrainDataset(merged_items, train=True)

        pin = (DEVICE.type == "cuda")
        # Windows / Notebook → num_workers=0, persistent_workers=False, tanpa prefetch_factor
        # Linux headless bisa dinaikkan, tapi default aman dibuat 0 agar tidak hang.
        dl = DataLoader(
            ds,
            batch_size=batch_size,
            shuffle=True,
            num_workers=0,                 # <<< KUNCI: single-process
            pin_memory=pin,
            drop_last=True,
            collate_fn=batch_collate
        )
        print(f"[RETRAIN] Loader: n={len(ds)} | batch={batch_size} | workers=0 | pin={pin}")
        return dl

    ft_loader = make_loader_for_fulltrain(train_items, val_items, batch_size=BATCH_SIZE)
    steps_per_epoch_ft = max(1, len(ft_loader))

    # ---------- Optimizer & Scheduler ----------
    ft_lr = LR * RETRAIN_LR_SCALE
    optimizer_ft = torch.optim.AdamW(model.parameters(), lr=ft_lr, weight_decay=WEIGHT_DECAY)
    scaler_ft = torch.cuda.amp.GradScaler(enabled=AMP_ENABLED)

    # OneCycle untuk awal (jika SWA_EPOCHS < RETRAIN_EPOCHS), lalu SWA untuk ekor
    total_steps_oc = max(0, (RETRAIN_EPOCHS - SWA_EPOCHS) * steps_per_epoch_ft)
    max_lr_ft = min(3e-4, ft_lr * 6)
    scheduler_oc = None
    if total_steps_oc > 0:
        scheduler_oc = torch.optim.lr_scheduler.OneCycleLR(
            optimizer_ft, max_lr=max_lr_ft, total_steps=total_steps_oc,
            pct_start=0.15, div_factor=10.0, final_div_factor=100.0,
            anneal_strategy="cos", three_phase=False
        )

    swa_model = AveragedModel(model).to(DEVICE)
    swa_start_epoch = max(1, RETRAIN_EPOCHS - SWA_EPOCHS + 1)
    swa_scheduler = SWALR(optimizer_ft, swa_lr=ft_lr)  # aman di-step per step (setelah optimizer.step)

    best_hold_mae = float("inf")
    no_improve = 0

    for e in range(1, RETRAIN_EPOCHS + 1):
        # pilih scheduler: OneCycle (awal) atau SWA (akhir)
        use_swa_phase = (e >= swa_start_epoch) or (scheduler_oc is None)
        sched = swa_scheduler if use_swa_phase else scheduler_oc

        # ---- TRAIN satu epoch (pakai train_one_epoch yang sudah ada) ----
        tr_loss, tr_align, tr_c, tr_loc, tr_ld = train_one_epoch(
            model, ft_loader, optimizer_ft, scaler_ft, e, RETRAIN_EPOCHS, sched
        )

        # ---- EVALUASI HOLDOUT untuk early stopping ----
        model.eval()
        hold_mae, hold_rmse = eval_holdout_mae(model)
        model.train()

        print(f"[RETRAIN {e:02d}/{RETRAIN_EPOCHS}] "
              f"loss={tr_loss:.4f} (align={tr_align:.4f}, c={tr_c:.2f}, loc={tr_loc:.3f}, stab={tr_ld:.4f}) | "
              f"HOLDOUT MAE={hold_mae:.3f} RMSE={hold_rmse:.3f} | lr={current_lr(optimizer_ft):.2e}")

        # SWA parameter averaging pada fase SWA
        if use_swa_phase:
            swa_model.update_parameters(model)

        # simpan best berbasis holdout
        if hold_mae < best_hold_mae:
            best_hold_mae = hold_mae
            no_improve = 0
            best_full_path = os.path.join(OUT_DIR, "best_fulltrain.pth")
            torch.save({"model": model.state_dict(), "retrained": True, "holdout_mae": hold_mae}, best_full_path)
            print(f"  -> [FT] Saved BEST (HOLDOUT MAE {best_hold_mae:.3f}) to {best_full_path}")
        else:
            no_improve += 1
            if no_improve >= FT_PATIENCE:
                print(f"[FT EarlyStopping] No improvement {FT_PATIENCE} epochs. Stop FT.")
                break

    # ---------- Recompute BN & simpan SWA ----------
    try:
        swa_model.eval()
        update_bn(ft_loader, swa_model, device=DEVICE)
    except Exception as _e:
        print("[SWA] update_bn skipped:", _e)

    best_full_path_swa = os.path.join(OUT_DIR, "best_fulltrain_swa.pth")
    torch.save({"model": swa_model.state_dict(), "retrained": True, "swa": True, "holdout_mae": best_hold_mae},
               best_full_path_swa)
    print(f"[SWA] Saved SWA full-train model to {best_full_path_swa}")



[RETRAIN] Fine-tuning on train+val with small LR + SWA + holdout early-stop ...
[RETRAIN] Start from: C:\Python\Template\Data Science\test\HOLOGY_UB_2025\out_csrnet_efficientb0\best_model.pth
[RETRAIN] Loader: n=1710 | batch=8 | workers=0 | pin=True


Train 1/4: 100%|██████████████████████████████████████████████████| 213/213 [12:14<00:00,  3.45s/it, align=0.1302, cnt=2
Hold(FT): 100%|██████████████████████████████████████████████████| 190/190 [00:10<00:00, 18.50it/s, gc=10.0, last_err=2.


[RETRAIN 01/4] loss=0.6958 (align=0.1302, c=21.50, loc=0.770, stab=0.0203) | HOLDOUT MAE=25.272 RMSE=79.519 | lr=5.00e-05
  -> [FT] Saved BEST (HOLDOUT MAE 25.272) to C:\Python\Template\Data Science\test\HOLOGY_UB_2025\out_csrnet_efficientb0\best_fulltrain.pth


Train 2/4: 100%|██████████████████████████████████████████████████| 213/213 [10:49<00:00,  3.05s/it, align=0.1270, cnt=2
Hold(FT): 100%|██████████████████████████████████████████████████| 190/190 [00:11<00:00, 16.56it/s, gc=10.0, last_err=3.


[RETRAIN 02/4] loss=0.7143 (align=0.1270, c=28.99, loc=0.933, stab=0.0241) | HOLDOUT MAE=25.825 RMSE=87.323 | lr=5.00e-05


Train 3/4: 100%|██████████████████████████████████████████████████| 213/213 [10:18<00:00,  2.90s/it, align=0.1334, cnt=2
Hold(FT): 100%|██████████████████████████████████████████████████| 190/190 [00:05<00:00, 37.44it/s, gc=10.0, last_err=3.


[RETRAIN 03/4] loss=0.6462 (align=0.1334, c=27.47, loc=0.972, stab=0.0257) | HOLDOUT MAE=24.315 RMSE=62.700 | lr=5.00e-05
  -> [FT] Saved BEST (HOLDOUT MAE 24.315) to C:\Python\Template\Data Science\test\HOLOGY_UB_2025\out_csrnet_efficientb0\best_fulltrain.pth


Train 4/4: 100%|██████████████████████████████████████████████████| 213/213 [08:32<00:00,  2.40s/it, align=0.1244, cnt=2
Hold(FT): 100%|██████████████████████████████████████████████████| 190/190 [00:06<00:00, 31.56it/s, gc=10.0, last_err=4.


[RETRAIN 04/4] loss=0.6474 (align=0.1244, c=26.15, loc=0.922, stab=0.0237) | HOLDOUT MAE=23.952 RMSE=65.588 | lr=5.00e-05
  -> [FT] Saved BEST (HOLDOUT MAE 23.952) to C:\Python\Template\Data Science\test\HOLOGY_UB_2025\out_csrnet_efficientb0\best_fulltrain.pth
[SWA] update_bn skipped: 'dict' object has no attribute 'to'
[SWA] Saved SWA full-train model to C:\Python\Template\Data Science\test\HOLOGY_UB_2025\out_csrnet_efficientb0\best_fulltrain_swa.pth


In [14]:
# ====== BLOK 12 — INFERENCE SEDERHANA (isi MODEL_PATH di bawah) ======
import os, math
import torch
import torch.nn.functional as F
import pandas as pd
from tqdm import tqdm

# --- 1) Tentukan checkpoint yang ingin dipakai ---
#    Isi dengan path lengkap ke file .pth kamu (best/last/retrained/custom)
MODEL_PATH = r"C:/Python/Template/Data Science/test/HOLOGY_UB_2025/out_csrnet_efficientb0/best_model.pth"  # <--- EDIT DI SINI

# --- 2) Muat model ---
print(f"[TEST] Loading weights from: {MODEL_PATH}")
ck = torch.load(MODEL_PATH, map_location=DEVICE)
sd = ck["model"] if isinstance(ck, dict) and "model" in ck else ck
model.load_state_dict(sd); model.eval()

# --- 3) Pengaturan prediksi ---
PRED_OFFSET = 0       # kalau ingin +1 ke semua prediksi, ubah ke 1
MIN_COUNT   = 1       # minimal hasil prediksi
TTA_SCALES  = [1.0]   # ubah ke [0.8, 1.0, 1.2] jika ingin TTA

# fallback tqdm style
def _bar(desc): 
    return dict(desc=desc, ncols=120, bar_format="{l_bar}{bar:50}{r_bar}")

# --- 4) DataLoader TEST sederhana (workers=0 agar aman di Windows/Notebook) ---
test_ds = CrowdTestDataset(TEST_IMG_DIR)
test_dl = DataLoader(
    test_ds, batch_size=1, shuffle=False,
    num_workers=0, pin_memory=(DEVICE.type=="cuda")
)

# --- 5) Fungsi infer satu batch (dengan/ tanpa TTA) ---
@torch.no_grad()
def infer_count(img_tensor: torch.Tensor) -> float:
    total = 0.0
    with torch.amp.autocast(device_type=("cuda" if DEVICE.type=="cuda" else "cpu"),
                            enabled=(DEVICE.type=="cuda")):
        for s in TTA_SCALES:
            if s == 1.0:
                pred = model(img_tensor.to(DEVICE, non_blocking=True)
                                   .to(memory_format=torch.channels_last))
                total += float(pred.detach().sum().item())
            else:
                _, C, H, W = img_tensor.shape
                Hs = int(round(H * s)); Ws = int(round(W * s))
                img_s = F.interpolate(img_tensor, size=(Hs, Ws), mode="bilinear", align_corners=False)
                pred = model(img_s.to(DEVICE, non_blocking=True)
                                  .to(memory_format=torch.channels_last))
                total += float(pred.detach().sum().item())
    return total / len(TTA_SCALES)

# --- 6) Loop prediksi & simpan submission ---
pred_rows = []
pbar = tqdm(test_dl, **_bar("Predict TEST"))
for batch in pbar:
    img  = batch["image"]
    meta = batch["meta"]

    cnt = infer_count(img)
    cnt = math.ceil(cnt + PRED_OFFSET)
    cnt = max(cnt, MIN_COUNT)

    image_id = meta["img_id"][0] if isinstance(meta["img_id"], list) else meta["img_id"]
    pred_rows.append((image_id, cnt))
    pbar.set_postfix(pred=f"{cnt}")
pbar.close()

sub_df = pd.DataFrame(pred_rows, columns=["image_id", "predicted_count"])

# nama file output mengikuti nama checkpoint biar mudah dilacak
base_name = os.path.splitext(os.path.basename(MODEL_PATH))[0]
sub_csv   = os.path.join(OUT_DIR, f"submission_{base_name}.csv")
sub_df.to_csv(sub_csv, index=False)
print(f"[TEST] Saved submission to {sub_csv}")

# (Opsional) Urutkan sesuai sample_submission.csv kalau ada
if os.path.exists(SAMPLE_SUB):
    try:
        samp   = pd.read_csv(SAMPLE_SUB)
        merged = samp[["image_id"]].merge(sub_df, on="image_id", how="left")
        merged["predicted_count"] = merged["predicted_count"].fillna(0).astype(int)
        sub_csv2 = os.path.join(OUT_DIR, f"submission_{base_name}_ordered.csv")
        merged.to_csv(sub_csv2, index=False)
        print(f"[TEST] Saved ordered submission to {sub_csv2}")
    except Exception as e:
        print(f"[WARN] Gagal buat versi ordered: {e}")


[TEST] Loading weights from: C:/Python/Template/Data Science/test/HOLOGY_UB_2025/out_csrnet_efficientb0/best_model.pth


Predict TEST: 100%|██████████████████████████████████████████████████| 500/500 [00:23<00:00, 20.93it/s, pred=10]  

[TEST] Saved submission to C:\Python\Template\Data Science\test\HOLOGY_UB_2025\out_csrnet_efficientb0\submission_best_model.csv
[TEST] Saved ordered submission to C:\Python\Template\Data Science\test\HOLOGY_UB_2025\out_csrnet_efficientb0\submission_best_model_ordered.csv
